In [21]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, confusion_matrix, accuracy_score, recall_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder
import datetime
import os
import json
import csv

In [22]:
# Parámetros
inputFolder = "1-input"
processFolder = "2-process"
outputFolder = "3-output"
logsFolder = "4-logs"

outputAlgorithmsFolder = outputFolder + r"\algorithms"
outputCrossValidationFolder = outputFolder + r"\cross-validation"

dataVisualizationTopLimit = 20

testSize = 0.25
randomState = 0
partitionsNumber = 5
samplingStrategy = 0.2 # Arreglar
percentileNumber = 90

In [23]:
# Funciones utilitarias
def readData(filePath, delimiter, encoding, header):
  data = pd.read_csv(filePath, delimiter=delimiter,encoding=encoding, header=header)
  return data

def writeJson(data, pathJson, encoding='utf-8'):
  with open(pathJson, 'w', encoding=encoding) as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

def writeCsv(data, pathCsv, encoding='utf-8'):
  with open(pathCsv, 'w', newline='', encoding=encoding) as f:
    if data:
      writer = csv.DictWriter(f, fieldnames=data[0].keys(), lineterminator='\n')
      writer.writeheader()
      writer.writerows(data)
    else:
      f.write("")

In [24]:
def preprocessingData(df):
  # Aca no se realizará ni parseo ni nada, ya vendrá desde la fuente
  # Solo se leerá, se dividirá y se entrenará

  # Balanceo de datos: Sobremuestreo aleatorio (oversampling)
  # De cada 5 no contratados habrá un contratado
  objectiveColumn = "contratado"
  dictResults = dict(df[objectiveColumn].value_counts().sort_index())
  #print(dictResults)

  maxKey = max(dictResults, key=dictResults.get)
  maxValue = max(dictResults.values())
  #print(maxKey)
  #print(maxValue)

  dfClassMaxKey = df[df[objectiveColumn] == maxKey]

  for key, value in dictResults.items():
    if key != maxKey:
      dfClass = df[df[objectiveColumn] == key]
      dfClassSampled = dfClass.sample(int(maxValue * samplingStrategy), random_state=randomState, replace=True)
      dfClassMaxKey = pd.concat([dfClassMaxKey, dfClassSampled],axis=0)

  df = dfClassMaxKey

  # Aleatorizacion del orden de los registros para evitar sesgos(filas)
  df = df.sample(frac = 1, random_state=randomState).reset_index(drop=True)  

  # Aplicando OrdinalEncoding a las variables categóricas ordinales()
  categoricalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "contratado" ] and columnType == "object" ]
  categoricalOrdinalColumns = [columnName for columnName in categoricalColumns if columnName in [ "estadoUltimoEstudio", "gradoUltimoEstudio" ]]

  encoder = OrdinalEncoder(categories=[[ "Abandonado", "En Curso", "Graduado" ]])
  encoder.fit(df[["estadoUltimoEstudio"]])
  df["estadoUltimoEstudio"] = encoder.transform(df[["estadoUltimoEstudio"]])

  encoder = OrdinalEncoder(categories=[[ "Otro", "Secundario", "Terciario/Tecnico", "Universitario", "Posgrado", "Master", "Doctorado" ]])
  encoder.fit(df[["gradoUltimoEstudio"]])
  df["gradoUltimoEstudio"] = encoder.transform(df[["gradoUltimoEstudio"]])

  #display(df)

  # Aplicando OneHotEncoding a las variables categóricas cardinales (transformación a numéricas mediante columnas)
  categoricalCardinalColumns = [columnName for columnName in categoricalColumns if columnName not in [ "estadoUltimoEstudio", "gradoUltimoEstudio" ]]
  for column in categoricalCardinalColumns:
    dummies = pd.get_dummies(df[[column]], prefix=column, dummy_na=True)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(columns=[column])

  #display(df.dtypes)

  # Aplicando MinMaxScaler a las variables numéricas (normalización) (esto tambien incluye a lastEducationStatus y lastEducationDegree, ya numéricas)
  # Algunas quedaran en 0.9999, esto porque no todas manejan la misma escala (sin decimales, o solo un decimal)
  numericalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "contratado" ] and columnType == "float64" ]
  for column in numericalColumns:
    df[column] = df[column].fillna(0.0)
  mms = MinMaxScaler()
  df[numericalColumns] = mms.fit_transform(df[numericalColumns])

  # Leyendo el numero de atributos
  # 19146 filas, 12403 columnas con el drop_first=True
  # 19146 filas, 12415 columnas sin el drop_first=True

  # Eliminando columnas con varianza cercana a cero (variables no afectan en el resultado del modelo)
  df.loc['std'] = df.std()
  stdArray = df.iloc[len(df)-1]
  ninetyNinthPercentile = np.percentile(stdArray, percentileNumber)
  df = df.transpose()
  df = df[df["std"]>ninetyNinthPercentile]
  df = df.transpose()
  df = df.drop(['std'], axis=0)

  # Lectura de las variables de características y objetivo
  objectiveColumn = "contratado"
  X = df.drop([objectiveColumn], axis=1)
  y = df[objectiveColumn]

  return X, y

In [25]:
def splitData(X, y, partitionNumber):
  # Dividiendo los dataframes de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSize, random_state=randomState)

  if partitionNumber != 0:
    # Obteniendo el total de filas del dataframe de testeo
    X_train, X_test, y_train, y_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)
    totalRows = len(X_train)

    # Determinando el límite inferior (primera fila) de la partición
    bottomLimit = int(totalRows*(1/partitionsNumber)*(partitionNumber-1)) + 1

    # Determinando el límite superior (última fila) de la partición
    topLimit = int(totalRows*(1/partitionsNumber)*partitionNumber)
    #print(bottomLimit, topLimit)

    # Determinando una lista con todas los numeros de las filas con la partición
    partitionList = [x for x in range(bottomLimit-1, topLimit)]

    # Determinando los dataframes de las categorías
    X_train, X_test = X_train[~X_train.index.isin(partitionList)], X_train[X_train.index.isin(partitionList)]

    # Determinando los dataframes de los objetivos
    y_train, y_test = y_train[~y_train.index.isin(partitionList)], y_train[y_train.index.isin(partitionList)]

  return X_train, X_test, y_train, y_test

In [26]:
def createClassifier(modelName):
  if modelName == "KNN":
    return KNeighborsClassifier()
  elif modelName == "LR":
    return LogisticRegression(random_state=randomState, max_iter=200)
  elif modelName == "GNB":
    return GaussianNB()
  elif modelName == "DT":
    return DecisionTreeClassifier(random_state=randomState)
  elif modelName == "SVM":
    return SVC(random_state=randomState)
  elif modelName == "RF":
    return RandomForestClassifier(random_state=randomState)
  elif modelName == "GB":
    return GradientBoostingClassifier(random_state=randomState)
  else:
    return KNeighborsClassifier()

In [27]:
def trainModel(X_train, X_test, y_train, y_test, modelName):
  # Creación del clasificador KNN
  clf = createClassifier(modelName)

  # Entrenamiento del clasificador KNN
  clf.fit(X_train, y_train)

  # Calculando la predicción del modelo con la data de prueba
  y_pred = clf.predict(X_test)

  return y_test, y_pred

In [28]:
def getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, algorithm = "KNN"):
  trainRows = len(y_train)
  testRows = len(y_test)

  # Calculando la exactitud del modelo
  accuracy = accuracy_score(y_test, y_pred)

  # Calculando la precisión del modelo
  precision = precision_score(y_test, y_pred)

  # Calculando la sensibilidad del modelo
  recall = recall_score(y_test, y_pred)

  # Calculando el valor F del modelo (robustez)
  f1Score = f1_score(y_test, y_pred)

  # Calculando el promedio de métricas
  metricsList = [accuracy, precision, recall, f1Score]
  metricsMean = sum(metricsList) / len(metricsList)

  # Calculando el tiempo de ejecución del modelo
  executionTime = (endDate - startDate).total_seconds()
  formatExecutionTime = "{:.2f}".format(executionTime) + "s"
  formatAverageTime = "{:.2f}".format(executionTime*1000/(trainRows + testRows)) + "ms"

  confussionMatrix = str(confusion_matrix(y_test, y_pred).tolist())
  
  return {
    "algoritmo": algorithm,
    "particion": "Total de datos" if partitionNumber == 0 else "Particion " + str(partitionNumber),
    "registrosEntrenamiento": trainRows,
    "registrosPrueba": testRows,
    "proporcionSobremuestreo": samplingStrategy,
    "tiempoEjecucion": formatExecutionTime,
    "matrizConfusion": confussionMatrix,
    "exactitud": "{:.2%}".format(accuracy),
    "precision": "{:.2%}".format(precision),
    "sensibilidad": "{:.2%}".format(recall),
    "robustez": "{:.2%}".format(f1Score),
    "promedioMetricas": "{:.2%}".format(metricsMean),
    "tiempoPromedio": formatAverageTime
  }


In [29]:
def main():
  # Definiendo el inicio del proceso
  startTime = datetime.datetime.now()
  print("Inicio: " + str(startTime))
  print()

  # Leyendo la data
  data = readData(os.path.join(inputFolder, "result.csv"), ',', 'utf-8', 0)

  # Determinando los dataframes de las categorías (X) y el objetivo (y)
  X, y = preprocessingData(data)

  # Definiendo los algoritmos a usar
  algorithms = ["KNN", "LR", "GNB", "DT", "SVM", "RF", "GB"]

  # Creando el arreglo de metricas de cada algoritmo
  algorithmsMetricsList = []

  for algorithm in algorithms:
    # Mostrando que algoritmo se usa
    print("Ejecutando para algoritmo {}".format(algorithm))

    # Inicio de ejecución
    startDate = datetime.datetime.now()
    #print("Inicio: " + str(startDate))

    # Separando data para el entrenamiento y testeo
    X_train, X_test, y_train, y_test = splitData(X, y, 0)

    # Realizar entrenamiento del modelo
    y_test, y_pred = trainModel(X_train, X_test, y_train, y_test, algorithm)

    # Fin de ejecución del modelo
    endDate = datetime.datetime.now()
    #print("Fin: " + str(endDate))
    #print("Tiempo: " + str(endDate-startDate))
    #print()

    # Obteniendo las métricas de la partición del modelo
    algorithmMetrics = getMetrics(y_train, y_test, y_pred, startDate, endDate, 0, algorithm)

    # Añadiendo la métrica de la partición a la lista de métricas
    algorithmsMetricsList.append(algorithmMetrics)

  # Ordenando las métricas
  algorithmsMetricsList = sorted(algorithmsMetricsList, key=lambda x: (x["algoritmo"]))

  # Escribiendo las metricas en un archivo de salida
  outputAlgorithmsDateTime = datetime.datetime.now()
  writeCsv(algorithmsMetricsList, os.path.join(outputAlgorithmsFolder, outputAlgorithmsDateTime.strftime("%Y-%m-%d %H-%M-%S") + ".csv"))

  # Elegir el algoritmo con mayor promedio de métricas
  maxAverageAlgorithm = max(algorithmsMetricsList, key=lambda x:x["promedioMetricas"])
  selectedAlgorithm, selectedAlgorithmAverage = maxAverageAlgorithm["algoritmo"], maxAverageAlgorithm["promedioMetricas"]

  print()
  print("Algoritmo con mayor promedio: {}".format(selectedAlgorithm))
  print("Promedio de métricas: {}".format(selectedAlgorithmAverage))
  print()

  # Creando el arreglo de metricas de la validación cruzada
  crossValidationMetricsList = []

  for partitionNumber in range (0, partitionsNumber+1):
    # Mostrando el número de la partició
    print("Ejecutando para {}".format("el total de datos" if partitionNumber == 0 else ("la partición {}".format(partitionNumber))))

    # Inicio de ejecución
    startDate = datetime.datetime.now()
    #print("Inicio: " + str(startDate))

    # Separando data para el entrenamiento y testeo
    X_train, X_test, y_train, y_test = splitData(X, y, partitionNumber)

    # Realizar entrenamiento del modelo
    y_test, y_pred = trainModel(X_train, X_test, y_train, y_test, selectedAlgorithm)

    # Fin de ejecución del modelo
    endDate = datetime.datetime.now()
    #print("Fin: " + str(endDate))
    #print("Tiempo: " + str(endDate-startDate))
    #print()

    # Obteniendo las métricas de la partición del modelo
    crossValidationMetrics = getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, selectedAlgorithm)

    # Añadiendo la métrica de la partición a la lista de métricas
    crossValidationMetricsList.append(crossValidationMetrics)

  # Ordenando las métricas
  crossValidationMetricsList = sorted(crossValidationMetricsList, key=lambda x: (x["particion"]))

  # Escribiendo las metricas en un archivo de salida
  outputCrossValidationDateTime = datetime.datetime.now()
  writeCsv(crossValidationMetricsList, os.path.join(outputCrossValidationFolder, outputCrossValidationDateTime.strftime("%Y-%m-%d %H-%M-%S") + ".csv"))

  # Elegir la partición con mayor promedio de métricas
  maxAveragePartition = max(algorithmsMetricsList, key=lambda x:x["promedioMetricas"])
  selectedPartition, selectedPartitionAverage = maxAveragePartition["particion"], maxAveragePartition["promedioMetricas"]

  print()
  print("Partición con mayor promedio: {}".format(selectedPartition))
  print("Promedio de métricas: {}".format(selectedPartitionAverage))
  print()
  print("Exactitud: {}".format(maxAveragePartition["exactitud"]))
  print("Precisión: {}".format(maxAveragePartition["precision"]))
  print("Sensibilidad: {}".format(maxAveragePartition["sensibilidad"]))
  print("Robustez: {}".format(maxAveragePartition["robustez"]))
  print("Tiempo promedio: {}".format(maxAveragePartition["tiempoPromedio"]))
  print()

  # Definiendo el fin del proceso
  endTime = datetime.datetime.now()
  print("Fin: " + str(endTime))
  print("Tiempo: " + str(endTime-startTime))

In [30]:
if __name__ == "__main__":
  main()

Inicio: 2023-06-06 02:35:51.374844

Ejecutando para algoritmo KNN
Ejecutando para algoritmo LR
Ejecutando para algoritmo GNB
Ejecutando para algoritmo DT
Ejecutando para algoritmo SVM
Ejecutando para algoritmo RF
Ejecutando para algoritmo GB

Algoritmo con mayor promedio: RF
Promedio de métricas: 98.30%

Ejecutando para el total de datos
Ejecutando para la partición 1
Ejecutando para la partición 2
Ejecutando para la partición 3
Ejecutando para la partición 4
Ejecutando para la partición 5

Partición con mayor promedio: Total de datos
Promedio de métricas: 98.30%

Exactitud: 99.34%
Precisión: 100.00%
Sensibilidad: 95.94%
Robustez: 97.93%
Tiempo promedio: 0.45ms

Fin: 2023-06-06 02:37:00.999676
Tiempo: 0:01:09.624832
